# Test Suites and Reports for Bicycle Demand Prediction

In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io

from datetime import datetime, time
from sklearn import datasets, ensemble

from evidently import ColumnMapping

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset

from evidently.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset, RegressionTestPreset
from evidently.tests import TestCategoryCount, TestCategoryShare, TestColumnDrift, TestColumnValueMean

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Bicycle Demand Data

In [ ]:
content = requests.get("https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip").content
with zipfile.ZipFile(io.BytesIO(content)) as arc:
    raw_data = pd.read_csv(arc.open("hour.csv"), header=0, sep=',', parse_dates=['dteday'], index_col='dteday')

In [ ]:
raw_data.index = raw_data.apply(
    lambda row: datetime.combine(row.name, time(hour=int(row['hr']))), axis = 1)

In [ ]:
raw_data.head()

In [ ]:
reference = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00']
current = raw_data.loc['2011-01-29 00:00:00':'2011-02-28 23:00:00']

In [ ]:
reference.head()

## Reports

In [ ]:
prediction = 'cnt'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']

In [ ]:
column_mapping = ColumnMapping()

column_mapping.prediction = prediction
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

In [ ]:
data_drift_report = Report(
    metrics=[DataDriftPreset()]
)

data_drift_report.run(reference_data=reference, current_data=current, column_mapping=column_mapping)

In [ ]:
data_drift_report.show(mode='inline')

## Test Suites

In [ ]:
drift_test = TestSuite(tests=[
    DataDriftTestPreset()
])

drift_test.run(reference_data=reference, 
               current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
               column_mapping=column_mapping)

drift_test.show(mode='inline')

In [ ]:
drift_test = TestSuite(tests=[
    TestColumnDrift(column_name=prediction),
    TestColumnDrift(column_name='temp'),
    TestColumnDrift(column_name='hum', stattest='wasserstein'),
    TestColumnDrift(column_name='windspeed', stattest='wasserstein', stattest_threshold=0.5)
])

drift_test.run(reference_data=reference, 
               current_data=current.loc['2011-01-29 00:00:00':'2011-02-07 23:00:00'],
               column_mapping=column_mapping)
drift_test.show(mode='inline')

In [ ]:
drift_test.json()

In [ ]:
drift_test.save_html('test_suite.html')

In [ ]:
drift_test.save_json('test_suite.json')

In [ ]:
drift_test.save('snapshot.json')

In [ ]:
loaded_test_suite = TestSuite.load('snapshot.json')

In [ ]:
loaded_test_suite.show(mode='inline')

## Cloud Dashboard

In [ ]:
from evidently.ui.workspace.cloud import CloudWorkspace

In [ ]:
ws = CloudWorkspace(
    token="dG9rbgGZMqXyO1NPcqGVQfOaF89zAqoxN6A5k7ozAeNKDs2t+wBQq9ZCa7IXB8v2jbg2KAK/YOYivgq6ftXaqYHpfszbBvBDP+zD/3rhwOiSlAaRFHD3rqqjOlhRYDKO2my/4WyDK3/K9AyvegdfCkCvcpP1I0RLymZI",
    url="https://app.evidently.cloud")

#to create project
project = ws.create_project("Bike Demand forecast [SHARED]")
project.description = "A demo project using Bike Demand forecasting dataset"

In [ ]:
project.dashboard.add_panel(
    DashboardPanelCounter(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        agg=CounterAgg.NONE,
        title="Bike Rental Demand Forecast: Data Drift",
    ),
    tab="Data Drift" 
)
project.dashboard.add_panel(
    DashboardPanelPlot(
        title="Share of drifting features",
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        values=[
            PanelValue(
                metric_id="DatasetDriftMetric",
                field_path="share_of_drifted_columns",
                legend="share",
            ),
        ],
        plot_type=PlotType.LINE,
        size=WidgetSize.HALF,
    ),
    tab="Data Drift"
)
project.dashboard.add_panel(
    DashboardPanelTestSuite(
        title="Data drift per column in time",
        test_filters=[
            TestFilter(test_id="TestColumnDrift", test_args={}),
        ],
        filter=ReportFilter(metadata_values={}, tag_values=[], include_test_suites=True),
        size=WidgetSize.FULL,
        panel_type=TestSuitePanelType.DETAILED,
        time_agg="1D",
    ),
    tab="Data Drift"
)

In [ ]:
project.add_snapshot(data_drift_report)

# Support Evidently
Enjoyed the tutorial? Star Evidently on GitHub to contribute back! This helps us continue creating free open-source tools for the community. https://github.com/evidentlyai/evidently